 ## Первый раздел.
 ### Импорт библиотек, подключение CUDA devices

In [1]:
!pip install numba
!pip install py-cpuinfo
from datetime import datetime
from tabulate import tabulate
from datetime import datetime
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
from tabulate import tabulate

import matplotlib.pyplot as plt
import numpy as np
import cpuinfo
import math
import matplotlib.pyplot as plt
import numba
import pandas as pd
import time

     |████████████████████████████████| 99 kB 3.4 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=4ce659b05ff662b0cc35a721097ca3ab4f6acdf8055c4dd3eb8c01d7657c6524
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
Successfully built py-cpuinfo


### *Проверяем, какое оборудование у нас подключенно.*:

In [2]:
print('----------------------------------------------------------------------')
print('CPU, выделенный на сервере Colaboratory:', cpuinfo.get_cpu_info()['brand_raw'])
print('GPU, выделенный на сервере Colaboratory:', cuda.get_current_device())
print('----------------------------------------------------------------------')
numba.cuda.detect()

----------------------------------------------------------------------
CPU, выделенный на сервере Colaboratory: Intel(R) Xeon(R) CPU @ 2.30GHz
GPU, выделенный на сервере Colaboratory: <CUDA device 0 'b'Tesla K80''>
----------------------------------------------------------------------
Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

### Второй раздел.

Функции вычисления на CPU и GPU <br/>

In [3]:
TPB = 16
number_iterations = 16
iterations = 50

In [4]:
def CPU_Pi_Calc(number_iterations, N):
    res = np.zeros(number_iterations)
    for i in range(number_iterations):
        x = np.random.uniform(size=N)
        y = np.random.uniform(size=N)
        z = x ** 2 + y ** 2 <= 1
        res[i] = 4.0 * sum(z) / N
    return res

In [5]:
@cuda.jit 
def GPU_Pi_Calc(res, rng):
  i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  h = 0
  if i < len(rng):
    x = xoroshiro128p_uniform_float32(rng, i)
    y = xoroshiro128p_uniform_float32(rng, i)
    if x ** 2 + y ** 2 <= 1.0:
      h += 1
  cuda.atomic.add(res, 0, h)

In [6]:
def DeviceHost(N):

  rng_states = create_xoroshiro128p_states(N, seed=1)
  res = [0]

  dev_res = cuda.to_device(res)
  GPU_Pi_Calc[N, TPB](dev_res, rng_states)
  tmp_ = dev_res.copy_to_host()
  pi = 4 * tmp_[0] / N
  return pi

In [7]:
def main(iterations , N):
  time_CPU = time.time()

  for i in range(iterations):
    pi_CPU = CPU_Pi_Calc(number_iterations, N).mean()
    
  time_CPU = (time.time() - time_CPU)
  
  time_GPU = time.time()

  for i in range(iterations):
    pi_GPU = DeviceHost(N)
    N = N + 10000
    
  time_GPU = (time.time() - time_GPU)
  
  return time_CPU, time_GPU, pi_CPU, pi_GPU


### Третий раздел.

*Выходные данные* <br/>

In [8]:
time_CPU, time_GPU, pi_CPU, pi_GPU = main(iterations, 10000)


print(f"Количество итераций: {iterations}")
print('----------------------------------------------------------------------')
print(f"Время вычисления на CPU: {round(time_CPU,2)} second.")
print(f"Число Pi на CPU: {pi_CPU}")
print('----------------------------------------------------------------------')
print(f"Время вычисления на GPU: {round(time_GPU,2)} second.")
print(f"Число Pi на GPU: {pi_GPU}")
print('----------------------------------------------------------------------')

Количество итераций: 50
----------------------------------------------------------------------
Время вычисления на CPU: 19.25 second.
Число Pi на CPU: 3.146775
----------------------------------------------------------------------
Время вычисления на GPU: 5.42 second.
Число Pi на GPU: 3.144608
----------------------------------------------------------------------


In [9]:
timing_df = pd.DataFrame({'Device':['CPU', 'GPU'],
                          'Time(sec)':[(round(time_CPU,2)), (round(time_GPU,2))],
                          'Number_Pi':[f"{pi_CPU}", f"{pi_GPU}"]})
                                         
timing_df

,Device,Time(sec),Number_Pi
0,CPU,19.25,3.146775
1,GPU,5.42,3.144608
